<a href="https://colab.research.google.com/github/tataphani/b_pt/blob/master/SAS_R_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
! pip install saspy
! pip install pyreadstat

     |████████████████████████████████| 2.4 MB 5.2 MB/s 


In [21]:
! which java

/usr/bin/java


In [22]:
##First Import saspy using this then Enter the IOM user which is You can use either this user ID or your email address 
## (nkvsas4@gmail.com  Sas_R_Python_3),
## along with your SAS Profile password, to sign in to SAS OnDemand for Academics:
## https://welcome.oda.sas.com and user Login and then password is 
import saspy
sas = saspy.SASsession(iomhost=['odaws01-usw2.oda.sas.com', 'odaws02-usw2.oda.sas.com',
 'odaws03-usw2.oda.sas.com', 'odaws04-usw2.oda.sas.com'],
 java='/usr/bin/java', iomport=8591)

Using SAS Config named: default
Please enter the IOM user id: nkvsas4@gmail.com
Please enter the password for IOM user : ··········
SAS Connection established. Subprocess id is 3365



In [89]:
import saspy
import pandas as pd 
pd.set_option('display.max_columns' , None)

import pyreadstat
from IPython.display import HTML
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
saspy.__version__

In [ ]:
#sas1 = saspy.SASsession()

In [ ]:
## Mounting data from Google drive

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [50]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"
!ls "/content/drive/My Drive/datasets"
!ls "/content/drive/My Drive/datasets/adsl.xpt"
!ls "/content/drive/My Drive/datasets/ts.xpt"
!ls "/content/drive/My Drive/datasets/adae.xpt"
!ls "/content/drive/My Drive/datasets/adsl.sas7bdat"
!ls"/content/drive/My Drive/datasets/ADaM_spec.xlsx"

'Colab Notebooks'		   'Feb R-Ladies Seattle Q&A.gdoc'
 Computer_desktop.zip		   'Getting started.pdf'
 DataManipulationinR_beigners.pdf  'Help Center.gsite'
 datasets			    my_documents_zip.rar
adae.xpt	adsl.xpt	     remain		ts.xpt
ADaM_spec.xlsx	dm.xpt		     SDTM_spec.xlsx	ts_xpt_r.xpt
adsl.sas7bdat	new_head_ts_xpt.xpt  ts_sasdt.sas7bdat
'/content/drive/My Drive/datasets/adsl.xpt'
'/content/drive/My Drive/datasets/ts.xpt'
'/content/drive/My Drive/datasets/adae.xpt'
'/content/drive/My Drive/datasets/adsl.sas7bdat'
/bin/bash: ls/content/drive/My Drive/datasets/ADaM_spec.xlsx: No such file or directory


Way to load R into ipython

In [ ]:
## reading the .xpt files using the R language
# activate R magic
%load_ext rpy2.ipython

##The rpy2.ipython extension is already loaded. To reload it, use:
#%reload_ext rpy2.ipython

In [ ]:
%%R
install.packages("devtools")
install.packages('dplyr')
install.packages('haven')
install.packages('reticulate')
install.packages('admiral') #Clinical datasets#
install.packages("rio")
install.packages("SASxport")

In [ ]:
%%R
library(devtools)
library(dplyr)
library(admiral)
library(rio)
library(reticulate)
library(dplyr)
library(ggplot2)
library(admiral)
library(SASxport)

In [ ]:
%%R
#devtools::install_github("https://github.com/atorus-research/xportr.git")
#library(xportr) # https://github.com/atorus-research/xportr/blob/master/inst/specs/ADaM_spec.xlsx

In [ ]:
#@title Installing the packages if needed
%%R
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) {
  install.packages("R.utils")
  install.packages("rio")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

In [83]:
# A bit of imports


# Load in the r magic
%reload_ext rpy2.ipython
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><



### -----Lets make a simple SAS dataset------


In [ ]:
## First SAS program using sas.submitLST
sas.submitLST('data a ; x = 1 ; run;proc print data = a ; run;' )

In [ ]:
sas.submitLST("data class ; set sashelp.class (obs = 5);  run; proc print data=class; run;", method='listorlog') 


##### To see how a SASDATA object to see PYTHON convention 
#####'?' in front of the funtion and see the DOCSTRING

In [25]:
?sas.sasdata

##### To see the source code PYTHON convention '??' in front of the funtion   

In [ ]:
??sas.sasdata

In [ ]:
?sas.submitLST("data class ; set sashelp.class (obs = 5);  run; proc print data=class; run;", method='listorlog') 




---


#####Reading the external data using Python and input dataset for SASsession to use

---



In [26]:
#import pandas as pd
ad_sl = pd.read_csv("https://raw.githubusercontent.com/sas2r/clinical_fd/master/data-raw/inst/extdata/adsl.csv")
#ad_sl.head(ad_sl)
sasdf = sas.df2sd(ad_sl , 'sasdf')

##### Print first five observations of the SAS dataset

In [ ]:
sas.submitLST('proc print data = work.sasdf (obs = 5 keep = usubjid trt01p age visit1dt     rfstdtc     rfendtc  visnumen );run;', method='listorlog') 

##### Create a simple statistics of age by treatment arm using SAS-listorlog- this is the default as of V3.6.5. returns the LST,unless it's empty, then it returns the LOG instead 

In [ ]:
sas.submitLST('proc means data = work.sasdf ; class trt01p ; Var age ;  run;', method='listorlog') 

In [ ]:
# Create a simple statistics of age by treatment arm using Python
result = ad_sl.groupby('trt01p').agg({'age':[ 'mean', 'min', 'max']})
print(result)

##### Reading the external data using **R rio package** 

In [31]:
%%R
library(dplyr)
library(rio)
adsl_r <- rio::import ("https://raw.githubusercontent.com/sas2r/clinical_fd/master/data-raw/inst/extdata/adsl.csv")

##### Create a simple statistics of age by treatment arm using dPLYR

In [ ]:
# R way of creating the Basic statistics
%%R
summ_adsl_r  <- adsl_r %>%
                group_by(trt01p) %>%
                summarise (ave_age = mean(age , na.rm = TRUE),
                           med_age = median (age , na.rm = TRUE),
                           min_age = min(age , na.rm = TRUE),
                           max_age = max(age , na.rm = TRUE)
                            )
summ_adsl_r   

##### Admiral package is an ADaM in R Asset Library, It has couple of ADS based datasets that can used just by calling admiral:: 

In [ ]:
%%R
#VISIT1DT  IS NOT AVAILABLE IN ADMIRAL DATASET
names(admiral::adsl)
adsl_r <- admiral::adsl  %>% 
          select(USUBJID,AGE,DTHFL,TRT01P,TRTSDTM , SAFFL) 
head(adsl_r)

>####### Read SAS .xpt files using SAS Proc Copy This does not work as it is creating sasdataset outside SAS Envirnoment. Work in progress for this chunck

In [ ]:
sas.submitLST("Libname xptfile xport '/content/drive/My Drive/datasets/adsl.xpt' ;"
              "proc copy inlib = xptfile outlib = work ;  run;"
              ,
              method='listorlog') 

>>###### Read SAS .xpt files using python

In [ ]:
#import  pandas as pd
read_adsl_xpt_using_py = pd.read_sas('/content/drive/My Drive/datasets/adsl.xpt')
read_adsl_xpt_using_py.head()

# Summary statistics of all the columns using describe 
#read_adsl_xpt_using_py.describe(include = ['float' , 'category'])  # Will give general statistics of the var


>>##### Read SAS .xpt files using R 
(RIO package is new universal data reader package) 

In [ ]:
%%R
adsl_xpt_r <- rio::import("/content/drive/My Drive/datasets/adsl.xpt")
head(tibble(adsl_xpt_r))

>###### Way to read SAS dataset into R

In [ ]:
%%R
adsl_atorus <- haven::read_sas("/content/drive/My Drive/datasets/adsl.sas7bdat")
head(adsl_atorus)

### You can read .Xlsx file 

var_spec <- readxl::read_xlsx("/content/drive/My Drive/datasets/ADaM_spec.xlsx", sheet = "Variables") %>%
  dplyr::rename(type = "Data Type") %>%
  rlang::set_names(tolower)

  data_spec <- readxl::read_xlsx("/content/drive/My Drive/datasets/ADaM_spec.xlsx", sheet = "Datasets") %>%
  rlang::set_names(tolower) %>%
  dplyr::rename(label = "description")


>###### Creating R data frame 

In [55]:
%%R
ts_xpt_r <- rio::import("/content/drive/My Drive/datasets/ts.xpt")
head_ts_xpt <- head(ts_xpt_r)

>##### Way to see how a data frame created dput

In [ ]:
%%R
#dput(ts_xpt_r )
#dput(head_ts_xpt ,control = c('keepNA' , 'keepInteger' , 'niceNames' , 'showAttributes'))
## Way to Drop a variable
new_head_ts_xpt <- head_ts_xpt %>%
                   select(-STUDYID ) %>%
                   mutate(LONG_DOM_AIN_NAME = DOMAIN) %>%
                   relocate(LONG_DOM_AIN_NAME ,.before = DOMAIN)

new_head_ts_xpt                   

>##### Write it out a .Xpt file using SASxport, see the new variable NAME_OF_THE_DOMAIN (longer than 8 Char) SASxport default to V5


In [ ]:
%%R
write.xport (new_head_ts_xpt , file='/content/drive/MyDrive/datasets/ts_xpt_r.xpt')


#Check if it is written right#
ts_xpt_r <- rio::import("/content/drive/My Drive/datasets/ts_xpt_r.xpt")
ts_xpt_r

>##### Write out to .Xpt file using HAVEN package, see the new variable NAME_OF_THE_DOMAIN (longer than 8 Char) HAVEN package can used to write V8


In [79]:
%%R
tmp <- tempfile (fileext = ".xpt")
new_head_ts_xpt_long <- new_head_ts_xpt
haven::write_xpt (new_head_ts_xpt_long , "/content/drive/My Drive/datasets/new_head_ts_xpt_long.xpt", 
                  version = 8 , 
                  name = NULL  )


In [80]:
%%R
tmp <- tempfile (fileext = ".xpt")
new_head_ts_xpt_long <- new_head_ts_xpt
haven::write_xpt (new_head_ts_xpt_long , tmp, version = 8 , name = NULL  )
haven::read_xpt(tmp)

# A tibble: 6 × 6
  LONG_DOM_AIN_NAME DOMAIN TSSEQ TSPARMCD TSPARM                          TSVAL 
  <chr>             <chr>  <dbl> <chr>    <chr>                           <chr> 
1 TS                TS         1 ADDON    Added on to Existing Treatments Y     
2 TS                TS         1 AGEMAX   Planned Maximum Age of Subjects No ma…
3 TS                TS         1 AGEMIN   Planned Minimum Age of Subjects 50 ye…
4 TS                TS         1 AGESPAN  Age Group                       ADULT…
5 TS                TS         2 AGESPAN  Age Group                       ELDER…
6 TS                TS         1 TBLIND   Trial Blinding Schema           DOUBL…


In [ ]:
%%R
ts_xpt_r <- rio::import("/content/drive/My Drive/datasets/new_head_ts_xpt_long.xpt")
ts_xpt_r

> ###### Writing .XPT file using Pandas write_xport( ) function 

In [116]:
ts_xpt_using_py = pd.read_sas('/content/drive/My Drive/datasets/ts.xpt')
pd_data = pd.DataFrame(ts_xpt_using_py.head())
##Rename the columns
 
#pd_data.assign(LONG_DOM_AIN_NAME = pd_data['DOMAIN'] )
pd_data['LONG_DOM_AIN_NAME'] = pd_data['DOMAIN'] 
print(pd_data )
pd.set_option ("display.max_columns" , None)
pd.set_option ("display.max_rows" , None)

           STUDYID DOMAIN  TSSEQ    TSPARMCD  \
0  b'CDISCPILOT01'  b'TS'    1.0    b'ADDON'   
1  b'CDISCPILOT01'  b'TS'    1.0   b'AGEMAX'   
2  b'CDISCPILOT01'  b'TS'    1.0   b'AGEMIN'   
3  b'CDISCPILOT01'  b'TS'    1.0  b'AGESPAN'   
4  b'CDISCPILOT01'  b'TS'    2.0  b'AGESPAN'   

                               TSPARM              TSVAL LONG_DOM_AIN_NAME  
0  b'Added on to Existing Treatments'               b'Y'             b'TS'  
1  b'Planned Maximum Age of Subjects'      b'No maximum'             b'TS'  
2  b'Planned Minimum Age of Subjects'        b'50 years'             b'TS'  
3                        b'Age Group'   b'ADULT (18-65)'             b'TS'  
4                        b'Age Group'  b'ELDERLY (> 65)'             b'TS'  


In [121]:
### Writing it out as .XPT file 
path = "/content/drive/My Drive/datasets/ts_xpt_write_py_long.xpt" 
pyreadstat.write_xport(pd_data , path)

ts_xpt_using_py_short = ts_xpt_using_py.head()
path = "/content/drive/My Drive/datasets/ts_xpt_using_py_short.xpt" 
pyreadstat.write_xport(ts_xpt_using_py_short , path)


In [ ]:
read_ts_xpt_using_py_long = pd.read_sas('/content/drive/My Drive/datasets/ts_xpt_write_py_long.xpt')
print(read_ts_xpt_using_py_long)


In [124]:
read_ts_xpt_using_py_short = pd.read_sas('/content/drive/My Drive/datasets/ts_xpt_using_py_short.xpt')
print(read_ts_xpt_using_py_short)


ValueError: ignored

The above is new code that is working

In [60]:
%%R
df1 = c(5, 3, 3)
df2 = c(1, 3, 4)

In [61]:
%R -o df1 -o df2

In [ ]:
%%R
install.packages('reticulate')
library(reticulate)
#install.packages("googledrive")
#library("googledrive")
#drive_find()

In [63]:
# If ts not available we need to install . put other packages if you need.
%%R
packages <- c("googledrive", "googlesheets4")
if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

In [64]:
%%R
library("googledrive")
library("googlesheets4")

Attaching package: ‘googlesheets4’



    request_generate, request_make




In [65]:
# Find Google Sheets anf get the id of the first sheet 
%%R
#library("googledrive")
# drive_find(type = "xpt" )


NULL


In [67]:
%%R
adsl <- read_xpt('/content/drive/My Drive/datasets/adsl.xpt')

  could not find function "read_xpt"




Error in read_xpt("/content/drive/My Drive/datasets/adsl.xpt") : 
  could not find function "read_xpt"


RInterpreterError: ignored

In [ ]:
##https://stackoverflow.com/questions/66430209/running-rstudio-on-google-colab
##https://rpy2.github.io/doc/latest/html/introduction.html#r-packages
import rpy2.robjects as robjects

import rpy2
print(rpy2.__version__)

In [ ]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

In [ ]:
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

# R package names
packnames = ('ggplot2', 'hexbin' 'dplyr' 'xportr' , 'SASxport' )

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [ ]:
##https://songjoyce.medium.com/google-colab-101-using-both-r-python-f04eca03e6b5

%load_ext rpy2.ipython



In [ ]:
from rpy2.robjects.packages import importr
utils = importr('utils')

dataf = utils.read_csv('https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/'
                       'master/notebooks/data/california_cities.csv')

In [ ]:
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
dataf

In [ ]:
# activate R magic
%load_ext rpy2.ipython

In [ ]:
%%R
x <- 42
print(x)

In [ ]:
%%R
install.packages('dplyr')
install.packages('haven')
install.packages('ggplot2')


In [ ]:
%%R
data(package = "dplyr")

In [ ]:
import pandas as pd
pandasdf = pd.read_csv("/content/sample_data/california_housing_train.csv")
sasdf = sas.df2sd(pandasdf, 'sasdf')
sas.submitLST("proc print data=work.sasdf (obs=5);run;", method='listorlog')

In [ ]:


"""
Read five NHANES data files using Pandas and merge them into a 2d array.

This value can be exported:

Z : a Pandas data frame containing all the data
"""

## Data file names (the files are in ../Data)
FN = ["DEMO_F.XPT", "BMX_F.XPT", "BPX_F.XPT", "DR1TOT_F.XPT", "DR2TOT_F.XPT"]

In [ ]:
 
import numpy as np
import pandas as pd
 
 df = pandas.read_sas('filename.XPT')